The following code can be used to order and process via the [AOP_Strip_Processor](http://gbdxdocs.digitalglobe.com/docs/advanced-image-preprocessor) a list of images, as specified by their GBDX catalog ids.

In [ ]:
# Specify your credentials and create a gbdx interface

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

In [ ]:
# specify catalog ids
catalog_ids = ['1030010004C45A00', '104001001E9EF700']

# create order task
# it the images are not on GBDX, this task will order them from the DG factory
order = gbdx.Task('Auto_Ordering')
# pass both catalog ids as inputs - this will launch a batch workflow
order.inputs.cat_id = catalog_ids
# for this particular task, we need to set this flag to true
order.impersonation_allowed = True

# create preprocessing task and set input parameters
aop = gbdx.Task('AOP_Strip_Processor')
aop.inputs.data = order.outputs.s3_location.value
aop.inputs.bands = 'MS'
aop.inputs.enable_dra = False
aop.inputs.enable_pansharpen = False
aop.inputs.enable_acomp = True
aop.inputs.enable_tiling = False
aop.inputs.ortho_epsg = 'UTM'     # this setting is optional

# create two-task preprocessing workflow
preprocess_wf = gbdx.Workflow([order, aop])

# set output location to platform-stories/trial-runs/random_str within your bucket/prefix
from os.path import join
import uuid
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

# the two processed images will be stored under output_location
preprocess_wf.savedata(aop.outputs.data, output_location)

# execute
preprocess_wf.execute()

# check under this location for the output
print output_location

In [ ]:
# monitor status
preprocess_wf.status

It is sometimes the case that pansharpening an image with the default Dynamic Range Adjustment (DRA) of the [image preprocessor](https://gbdxdocs.digitalglobe.com/docs/advanced-image-preprocessor) results in poor viewing quality.
The solution is to apply DRA using Base Layer Matching. There is a specific task that does this and this is how it plugs into the image preprocessor. 

In [ ]:
from os.path import join

catid = '1040010014800C00'
    
order = gbdx.Task('Auto_Ordering')
order.inputs.cat_id = catid
order.impersonation_allowed = True

aop = gbdx.Task('AOP_Strip_Processor')
aop.inputs.data = order.outputs.s3_location.value
aop.inputs.enable_dra = False                          # disable automatic dra

blm = gbdx.Task('baselayermatch')
blm.inputs.data = aop.outputs.data.value
blm.inputs.cloud_id = catid                            # take clouds into account when dra'ing

preprocess_wf = gbdx.Workflow([order, aop, blm])
preprocess_wf.savedata(blm.outputs.data, join('platform-stories/trial-runs', catid))
preprocess_wf.execute()        